SOURCE : https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/

In [3]:
import pandas as pd
import requests
import numpy as np
import os
import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time


#from pcloud_dl import download_file
#pcloud-dl <link/code>

In [2]:
local_filename = "/Users/emmanuelnorvez/pCloud Drive/Data/Dataset/Immobilier_DVF/full.csv"
df_dvf = pd.read_csv(local_filename, sep=',')

/var/folders/95/c07zk0g150xgjccxhtynkw8w0000gn/T/ipykernel_24833/3023975462.py:2: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dvf = pd.read_csv(local_filename, sep=',')


In [13]:
df_dvf.tail()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
3458638,2024-1218061,2024-11-28,1,Vente,4550000.0,147.0,NaN,RUE DE CHARENTON,1794.0,75012,...,Appartement,23.0,2.0,S,sols,NaN,NaN,323.0,2.382241,48.845284
3458639,2024-1218061,2024-11-28,1,Vente,4550000.0,147.0,NaN,RUE DE CHARENTON,1794.0,75012,...,Appartement,23.0,2.0,S,sols,NaN,NaN,323.0,2.382241,48.845284
3458640,2024-1218061,2024-11-28,1,Vente,4550000.0,147.0,NaN,RUE DE CHARENTON,1794.0,75012,...,Appartement,23.0,2.0,S,sols,NaN,NaN,323.0,2.382241,48.845284
3458641,2024-1218061,2024-11-28,1,Vente,4550000.0,147.0,NaN,RUE DE CHARENTON,1794.0,75012,...,Appartement,23.0,2.0,S,sols,NaN,NaN,323.0,2.382241,48.845284
3458642,2024-1218061,2024-11-28,1,Vente,4550000.0,147.0,NaN,RUE DE CHARENTON,1794.0,75012,...,Local industriel. commercial ou assimilé,42.0,0.0,S,sols,NaN,NaN,323.0,2.382241,48.845284


# NETTOYAGE CODE POSTAL & LAT LONG

In [4]:
print("Vérification des colonnes latitude et longitude après nettoyage:")
print(df_dvf['latitude'].dtype)
print(df_dvf['longitude'].dtype)
print(df_dvf['latitude'].isnull().sum())
print(df_dvf['longitude'].isnull().sum())

Vérification des colonnes latitude et longitude après nettoyage:
float64
float64
39199
39199


In [5]:
df_dvf.dropna(subset=['latitude', 'longitude'], inplace=True)
print(df_dvf['latitude'].isnull().sum())
print(df_dvf['longitude'].isnull().sum())

0
0


In [6]:
# --- Nettoyage et conversion du Code Postal ---
print("Avant conversion du code_postal:")
print(df_dvf['code_postal'].head())
print(df_dvf['code_postal'].dtype)

# Convertir en numérique (les NaN restent des NaN)
# Puis convertir en Int64 (pour gérer les NaN correctement en tant qu'entiers)
# Enfin, convertir en chaîne de caractères
df_dvf['code_postal'] = pd.to_numeric(df_dvf['code_postal'], errors='coerce').fillna(-1).astype(int).astype(str)

# Si le code postal doit avoir une longueur fixe (ex: 5 chiffres en France), on peut rajouter des zéros
# Pour les codes postaux français qui commencent par 0 (ex: 01000), il faut s'assurer d'avoir 5 chiffres
df_dvf['code_postal'] = df_dvf['code_postal'].apply(lambda x: x.zfill(5) if x != '-1' else '')

# Si des -1 temporaires ont été utilisés, on peut les re-transformer en NaN ou les laisser vides
df_dvf['code_postal'] = df_dvf['code_postal'].replace('-1', '') # Ou pd.NA

print("\nAprès conversion du code_postal:")
print(df_dvf['code_postal'].head())
print(df_dvf['code_postal'].dtype)
print(f"Nombre de codes postaux vides/NaN après nettoyage: {df_dvf['code_postal'].isin(['', pd.NA]).sum()}")

Avant conversion du code_postal:
0    1230.0
1    1170.0
2    1290.0
3    1170.0
4    1170.0
Name: code_postal, dtype: float64
float64

Après conversion du code_postal:
0    01230
1    01170
2    01290
3    01170
4    01170
Name: code_postal, dtype: object
object
Nombre de codes postaux vides/NaN après nettoyage: 24079


In [8]:
import folium
from IPython.display import display # Pour afficher la carte dans un notebook

# --- ÉTAPE 1 : Afficher une carte simple pour choisir les coordonnées ---

print("--- ÉTAPE 1 : Choisissez votre point de recherche en cliquant sur la carte ---")
print("Une fois cliqué, un popup affichera les coordonnées (Latitude, Longitude).")
print("Notez ces coordonnées et utilisez-les dans la section de code suivante pour votre recherche.")

# Créer une carte Folium centrée par défaut sur la France ou une autre zone pertinente
# (Ces coordonnées ne chargent aucune donnée, elles définissent juste le centre initial de la carte)
initial_lat = 46.5 # Centre de la France
initial_lon = 2.5  # Centre de la France

m_selection = folium.Map(location=[initial_lat, initial_lon], zoom_start=6)

# Ajouter le plugin LatLngPopup pour afficher les coordonnées au clic
m_selection.add_child(folium.LatLngPopup())

# Afficher la carte dans le notebook
display(m_selection)

print("\n--- Après avoir cliqué et noté les coordonnées, insérez-les ici dans votre code de recherche : ---")
# input_lat = COORDONNEE_LATITUDE_COPIEE
# input_lon = COORDONNEE_LONGITUDE_COPIEE
# search_radius_meters = 300 # ou la valeur souhaitée


--- ÉTAPE 1 : Choisissez votre point de recherche en cliquant sur la carte ---
Une fois cliqué, un popup affichera les coordonnées (Latitude, Longitude).
Notez ces coordonnées et utilisez-les dans la section de code suivante pour votre recherche.



--- Après avoir cliqué et noté les coordonnées, insérez-les ici dans votre code de recherche : ---


In [ ]:
# --- Saisie des coordonnées et rayon (pour le test dans le notebook, vous les fixez ici) ---
# Coordonnées de test
input_lat = 48.1012
input_lon = -1.6697
search_radius_meters = 300

# Pour le test, définissez le code postal de la zone de recherche.
# Dans une application Streamlit, ce serait une entrée utilisateur.
# Pour le test avec Paris (48.8566, 2.3522), un code postal pertinent serait '75001' ou '75012' si votre point est dans le 12e arr.
# Pour Rennes (48.11, 1.68), le code postal serait '35000'.
central_point_postal_code = '35000' 

# --- NOUVEAU: Pré-filtrage par code postal ---
print(f"Filtrage initial par code postal : {central_point_postal_code}")
df_pre_filtered = df_dvf[df_dvf['code_postal'] == central_point_postal_code]

print(f"Nombre de mutations dans le code postal {central_point_postal_code} : {len(df_pre_filtered)}")

if df_pre_filtered.empty:
    print(f"Aucune mutation trouvée pour le code postal {central_point_postal_code}. La recherche géodésique ne sera pas effectuée.")
    # On continue mais df_filtered restera vide
    df_filtered = pd.DataFrame()
else:
    # --- Filtrage des mutations dans le rayon (uniquement sur le sous-ensemble) ---
    center_point = (input_lat, input_lon)
    mutations_in_radius = []
    
    # Itération sur le DataFrame pré-filtré, pas sur le DataFrame complet
    for index, row in df_pre_filtered.iterrows(): 
        mutation_point = (row['latitude'], row['longitude'])
        distance = geodesic(center_point, mutation_point).meters
        if distance <= search_radius_meters:
            mutations_in_radius.append(row)

    df_filtered = pd.DataFrame(mutations_in_radius)

print(f"\nNombre de mutations trouvées dans un rayon de {search_radius_meters} mètres (après pré-filtrage par code postal) : {len(df_filtered)}")
if not df_filtered.empty:
    print("\nPremières mutations filtrées:")
    print(df_filtered[['date_mutation', 'valeur_fonciere', 'adresse_nom_voie']].head())
else:
    print("\nAucune mutation trouvée dans ce rayon après le double filtre.")

# --- Affichage de la carte (le reste du code est inchangé) ---
m = folium.Map(location=[input_lat, input_lon], zoom_start=15)

# Ajouter le cercle de rayon
folium.Circle(
    location=center_point,
    radius=search_radius_meters,
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.1,
    tooltip=f"{search_radius_meters} mètres"
).add_to(m)

# Ajouter un marqueur pour le point central
folium.Marker(
    location=center_point,
    popup=f"Point central<br>Lat: {input_lat:.4f}<br>Lon: {input_lon:.4f}",
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# Ajouter les mutations filtrées sur la carte
if not df_filtered.empty:
    marker_cluster = MarkerCluster().add_to(m)
    for idx, row in df_filtered.iterrows():
        popup_html = f"""
        <b>Date:</b> {row['date_mutation']}<br>
        <b>Valeur:</b> {row['valeur_fonciere']}<br>
        <b>Adresse:</b> {row['adresse_numero'] if pd.notna(row['adresse_numero']) else ''} {row['adresse_nom_voie']}<br>
        <b>Code Postal:</b> {row['code_postal']}<br>
        <b>Type Local:</b> {row['type_local'] if pd.notna(row['type_local']) else 'N/A'}<br>
        <b>Surface:</b> {row['surface_reelle_bati'] if pd.notna(row['surface_reelle_bati']) else 'N/A'} m²
        """
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=folium.Popup(popup_html, max_width=300)
        ).add_to(marker_cluster)

# Afficher la carte dans le notebook
m

Filtrage initial par code postal : 35000
Nombre de mutations dans le code postal 35000 : 5802

Nombre de mutations trouvées dans un rayon de 300 mètres (après pré-filtrage par code postal) : 36

Premières mutations filtrées:
        date_mutation  valeur_fonciere      adresse_nom_voie
1218655    2024-01-15         500000.0     RUE ALBERT-MARTIN
1218656    2024-01-15         500000.0     RUE ALBERT-MARTIN
1219872    2024-01-11         492000.0  RUE GUSTAVE-FLAUBERT
1227078    2024-03-13         280000.0       RUE DES ORMEAUX
1227079    2024-03-13         280000.0       RUE DES ORMEAUX


In [12]:
df_filtered

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
1218655,2024-419802,2024-01-15,1,Vente,500000.0,10.0,NaN,RUE ALBERT-MARTIN,0180,35000,...,Maison,110.0,5.0,S,sols,NaN,NaN,302.0,-1.667151,48.101661
1218656,2024-419802,2024-01-15,1,Vente,500000.0,8.0,NaN,RUE ALBERT-MARTIN,0180,35000,...,NaN,NaN,NaN,S,sols,NaN,NaN,1.0,-1.667298,48.101689
1219872,2024-420287,2024-01-11,1,Vente,492000.0,2.0,NaN,RUE GUSTAVE-FLAUBERT,4070,35000,...,Maison,94.0,5.0,S,sols,NaN,NaN,224.0,-1.665930,48.100552
1227078,2024-423188,2024-03-13,1,Vente,280000.0,47.0,NaN,RUE DES ORMEAUX,6990,35000,...,Appartement,83.0,4.0,NaN,NaN,NaN,NaN,NaN,-1.668448,48.098866
1227079,2024-423188,2024-03-13,1,Vente,280000.0,47.0,NaN,RUE DES ORMEAUX,6990,35000,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,-1.668448,48.098866
1227080,2024-423188,2024-03-13,1,Vente,280000.0,47.0,NaN,RUE DES ORMEAUX,6990,35000,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,-1.668448,48.098866
1237035,2024-426778,2024-05-22,1,Vente,570000.0,15.0,NaN,RUE CHARLES-MULLER,1730,35000,...,Maison,114.0,5.0,S,sols,NaN,NaN,399.0,-1.671001,48.098753
1237036,2024-426778,2024-05-22,1,Vente,570000.0,15.0,NaN,RUE CHARLES-MULLER,1730,35000,...,Dépendance,NaN,0.0,S,sols,NaN,NaN,399.0,-1.671001,48.098753
1237037,2024-426778,2024-05-22,1,Vente,570000.0,15.0,NaN,RUE CHARLES-MULLER,1730,35000,...,Dépendance,NaN,0.0,S,sols,NaN,NaN,399.0,-1.671001,48.098753
1237038,2024-426778,2024-05-22,1,Vente,570000.0,15.0,NaN,RUE CHARLES-MULLER,1730,35000,...,Dépendance,NaN,0.0,S,sols,NaN,NaN,399.0,-1.671001,48.098753


In [4]:
local_filename = "/Users/emmanuelnorvez/pCloud Drive/Data/Dataset/Immobilier_DVF/ValeursFoncieres-2023.txt"
dvf_2023 = pd.read_csv(local_filename, sep='|')
dvf_2023

/var/folders/95/c07zk0g150xgjccxhtynkw8w0000gn/T/ipykernel_39087/3121333282.py:2: DtypeWarning: Columns (14,18,23,24,26,28,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  dvf_2023 = pd.read_csv(local_filename, sep='|')


,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2023,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2023,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2023,Vente,...,NaN,1,2.0,Appartement,NaN,233.0,8.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2023,Vente,...,NaN,0,1.0,Maison,NaN,64.0,3.0,S,NaN,988.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2023,Vente,...,NaN,0,1.0,Maison,NaN,73.0,3.0,S,NaN,835.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3806973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/10/2023,Adjudication,...,NaN,1,2.0,Appartement,NaN,16.0,1.0,NaN,NaN,NaN
3806974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21/09/2023,Adjudication,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
3806975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,15/09/2023,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
3806976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,15/09/2023,Vente,...,NaN,1,2.0,Appartement,NaN,80.0,4.0,NaN,NaN,NaN


Géocoder un fichier CSV : https://adresse.data.gouv.fr/outils/csv


In [5]:
dvf_2023.columns

Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')

In [6]:
print(dvf_2023['Voie'].isnull().sum())
print(dvf_2023['Code postal'].isnull().sum())

25785
25674


In [16]:
# Assurez-vous que dvf_2023 est déjà chargé avec :
# dvf_2023 = pd.read_csv(local_filename, sep='|', decimal=',', encoding='utf-8', on_bad_lines='warn', low_memory=False)

# --- Nettoyage et conversion de la colonne 'Code postal' pour dvf_2023 ---
print("Avant conversion du 'Code postal':")
# Vérifiez la colonne spécifique de votre DataFrame
print(dvf_2023['Code postal'].head())
print(dvf_2023['Code postal'].dtype)

# Convertir en numérique (les NaN restent des NaN si 'errors'='coerce')
# Puis remplacer les NaN par une valeur temporaire (-1) pour la conversion en int
# Convertir en entier, puis en chaîne de caractères
dvf_2023['Code postal'] = pd.to_numeric(dvf_2023['Code postal'], errors='coerce') \
                            .fillna(-1) \
                            .astype(int) \
                            .astype(str)

# Si le code postal doit avoir une longueur fixe (ex: 5 chiffres en France), on peut rajouter des zéros
# Pour les codes postaux français qui commencent par 0 (ex: 01000), il faut s'assurer d'avoir 5 chiffres
dvf_2023['Code postal'] = dvf_2023['Code postal'].apply(lambda x: x.zfill(5) if x != '-1' else '')

# Remplacer les -1 temporaires par des chaînes vides (ou pd.NA si vous préférez)
dvf_2023['Code postal'] = dvf_2023['Code postal'].replace('-1', '')

print("\nAprès conversion du 'Code postal':")
print(dvf_2023['Code postal'].head())
print(dvf_2023['Code postal'].dtype)
print(f"Nombre de codes postaux vides/NaN après nettoyage: {dvf_2023['Code postal'].isin(['', pd.NA]).sum()}")

Avant conversion du 'Code postal':
0    1630.0
1    1630.0
2    1630.0
3    1450.0
4    1800.0
Name: Code postal, dtype: object
object

Après conversion du 'Code postal':
0    01630
1    01630
2    01630
3    01450
4    01800
Name: Code postal, dtype: object
object
Nombre de codes postaux vides/NaN après nettoyage: 25674


In [17]:
# Assurez-vous que votre DataFrame 'df' est bien chargé à partir de dvf_2023
# (Je pars du principe que dvf_2023 a été lu avec succès avec le bon séparateur '|' et decimal=',')

# Pour éviter les problèmes de casse ou d'espaces superflus
# Convertir la colonne 'Voie' en string, la mettre en majuscules et enlever les espaces inutiles.
dvf_2023['Voie'] = dvf_2023['Voie'].astype(str).str.strip().str.upper()

# Filtrer le DataFrame pour trouver les lignes où 'Voie' est 'RUE LE CHAPELIER'
# Note : Il est important de bien nettoyer la chaîne de caractères recherchée aussi
rue_recherchee = "RUE LE CHAPELIER".strip().upper()
lignes_rue_le_chapelier = dvf_2023[dvf_2023['Voie'] == rue_recherchee]

# Afficher les premières lignes du résultat
print(f"Nombre de mutations trouvées pour '{rue_recherchee}' : {len(lignes_rue_le_chapelier)}")
if not lignes_rue_le_chapelier.empty:
    print("\nPremières lignes trouvées pour 'RUE LE CHAPELIER' :")
    print(lignes_rue_le_chapelier.head())
else:
    print(f"Aucune ligne trouvée pour la rue '{rue_recherchee}'.")

Nombre de mutations trouvées pour 'RUE LE CHAPELIER' : 0
Aucune ligne trouvée pour la rue 'RUE LE CHAPELIER'.


In [18]:
# Assurez-vous que votre DataFrame 'dvf_2023' est bien chargé avec le bon séparateur '|' et decimal=','
# comme nous l'avons défini précédemment.

# --- Nettoyage et standardisation de la colonne 'Code postal' ---
# C'est une bonne pratique de s'assurer que la colonne est en string et nettoyée,
# même si le read_csv devrait l'avoir bien gérée si elle est numérique dans le fichier source.
# Remplir les codes postaux de 4 chiffres avec un 0 initial, si nécessaire.
dvf_2023['Code postal'] = dvf_2023['Code postal'].astype(str).str.strip().str.zfill(5)


# --- Filtre sur le Code postal ---
code_postal_recherche = "35000"
lignes_code_postal_35000 = dvf_2023[dvf_2023['Code postal'] == code_postal_recherche]

# Afficher les premières lignes du résultat
print(f"Nombre de mutations trouvées pour le code postal '{code_postal_recherche}' : {len(lignes_code_postal_35000)}")

if not lignes_code_postal_35000.empty:
    print(f"\nPremières lignes trouvées pour le code postal '{code_postal_recherche}' :")
    # Affiche quelques colonnes pertinentes pour vérifier les données
    print(lignes_code_postal_35000[['Date mutation', 'Valeur fonciere', 'Voie', 'Commune', 'Code postal']].head())
else:
    print(f"Aucune ligne trouvée pour le code postal '{code_postal_recherche}'.")

Nombre de mutations trouvées pour le code postal '35000' : 6315

Premières lignes trouvées pour le code postal '35000' :
        Date mutation Valeur fonciere                   Voie Commune  \
1363687    04/01/2023       160000,00                D ANJOU  RENNES   
1363688    04/01/2023       160000,00                D ANJOU  RENNES   
1363697    06/01/2023       120000,00  DE LA TOUR D AUVERGNE  RENNES   
1363698    06/01/2023       120000,00  DE LA TOUR D AUVERGNE  RENNES   
1363699    06/01/2023       120000,00  DE LA TOUR D AUVERGNE  RENNES   

        Code postal  
1363687       35000  
1363688       35000  
1363697       35000  
1363698       35000  
1363699       35000  


In [19]:
dvf_2023['Code postal']

0          01630
1          01630
2          01630
3          01450
4          01800
           ...  
3806973    75015
3806974    75006
3806975    75016
3806976    75016
3806977    75012
Name: Code postal, Length: 3806978, dtype: object